In [1]:
import json
import base64
from PIL import Image
from io import BytesIO
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
import boto3

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [2]:
#Set variables
session = sagemaker.Session()
region = session.boto_region_name
account_id = session.account_id()

print("Region:", region)
print("Account:", account_id)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Region: us-east-1
Account: 610912512102


In [3]:
sm_client = boto3.client("sagemaker")

model_name = "forabot-nerf-model"

# This will fail if model doesn't exist, so we wrap in try/except
try:
    sm_client.delete_model(ModelName=model_name)
    print(f"Deleted existing model: {model_name}")
except sm_client.exceptions.ClientError as e:
    print(f"Could not delete model (maybe it doesn't exist): {e}")

Could not delete model (maybe it doesn't exist): An error occurred (AccessDeniedException) when calling the DeleteModel operation: User: arn:aws:sts::610912512102:assumed-role/datazone_usr_role_43l9xaxd7j28pz_504bhe16x4mjzb/SageMaker is not authorized to perform: sagemaker:DeleteModel on resource: arn:aws:sagemaker:us-east-1:610912512102:model/forabot-nerf-model because no identity-based policy allows the sagemaker:DeleteModel action


In [4]:
#Deploy as a real time endpoint
#This will:
#Create the Model
#Create the EndpointConfig
#Create the Endpoint

In [5]:
role = get_execution_role()
version = "sm-v17"

image_uri = f"610912512102.dkr.ecr.us-east-1.amazonaws.com/sbaner24:{version}"
model_data = "s3://amazon-sagemaker-610912512102-us-east-1-0379069f3fe5/dzd_45z6d3gc7k13dz/43l9xaxd7j28pz/shared/data/nerfstudio-data/processed_forabot_BN_final-large2_2025-06-11_122746.tar.gz"
model_name = f"forabot-nerf-model-{version}"
endpoint_name = f"forabot-nerf-endpoint-{version}"

nerf_model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=role,
    name=model_name,
    sagemaker_session=session,
    env={"MODEL_DIR": "/opt/ml/model"},
)

predictor = nerf_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name,
)


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
------------------------------------------------------*

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 │   env={"MODEL_DIR": "/opt/ml/model"},                                                     │
│   16 )                                                                                           │
│   17                                                                                             │
│ ❱ 18 predictor = nerf_model.deploy(                                                              │
│   19 │   initial_instance_count=1,                                                               │
│   20 │   instance_type="ml.g4dn.xlarge",                                                         │
│   21 │   endpoint_name=endpoint_name,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:1814 in deploy                        │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:6250 in                             │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                                                      │
│   6251 │   │   │   endpoint_name=name,                                                           │
│   6252 │   │   │   config_name=name,                                                             │
│   6253 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:5095 in create_endpoint             │
│                                                                                                  │
│   5092 │   │   │   logger.error(                                                                 │
│   5093 │   │   │   │   "Please check the troubleshooting guide for common errors: %s", troubles  │
│   5094 │   │   │   )                                                                             │
│ ❱ 5095 │   │   │   raise e                                                                       │
│   5096 │                                                                                         │
│   5097 │   def endpoint_in_service_or_not(self, endpoint_na